In [1]:
import torch
import torch.nn as nn
import sys
from torch.utils.data import DataLoader
from datetime import datetime

sys.path.append("C:/Users/yuuta/Documents/fashion")
from model_learning.model_structure.image_encoder import  ImageEncoderV2
from model_learning.model_structure.dataset import TripletDataset

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
dataset = TripletDataset('C:/Users/yuuta/Documents/fashion/model_learning/versatility/data/triplet_n_pair_train.csv', limit=5000000)

In [4]:
from torch.utils.data import DataLoader
LEARNING_LATE = 1e-4
BATCH_SIZE = 32
EPOCHS = 10
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [5]:

model = ImageEncoderV2().to(device)
# 
# model.load_state_dict(torch.load('C:/Users/yuuta/Documents/fashion/model_learning/versatility/models/triplet-image-2024-01-09.pth'))
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_LATE)

loss_fn = nn.TripletMarginLoss(margin=20.0, p=2, eps=1e-7) # , reduction='sum'

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (anchor, positive, negative) in enumerate(dataloader):        
        optimizer.zero_grad()

        anchor_vecter = model(anchor.to(device))
        positive_vecter = model(positive.to(device))
        negative_vecter = model(negative.to(device))

        loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)

        # バックプロパゲーション
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item() / BATCH_SIZE, batch * BATCH_SIZE
            print(f"loss: {loss:>15.12f}  [{current:>5d}/{size:>5d}]")

        if batch % 3000 == 0:
            now = datetime.now()
            formatted_date = now.strftime("%Y-%m-%d")

            torch.save(model.state_dict(), f'C:/Users/yuuta/Documents/fashion/model_learning/versatility/models/triplet-image-{formatted_date}.pth')


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss = 0
    p_d = 0
    n_d = 0
    with torch.no_grad():
        for (anchor, positive, negative) in dataloader:
            anchor_vecter = model(anchor.to(device))
            positive_vecter = model(positive.to(device))
            negative_vecter = model(negative.to(device))
            # print(anchor_vecter.shape)
            positive_d = torch.norm(anchor_vecter - positive_vecter, dim=1, keepdim=True)
            negative_d = torch.norm(anchor_vecter - negative_vecter, dim=1, keepdim=True)
            # print('=====positive_d====')
            # print(torch.norm(anchor_vecter[0] - positive_vecter[0]))
            # print('=====negative_d====')
            # print(torch.norm(anchor_vecter[0] - negative_vecter[0]))

            p_d += torch.sum(positive_d)
            n_d += torch.sum(negative_d)
            loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)

            test_loss += loss.item()
            
    test_loss /= size
    print(f"Avg loss: {test_loss:>15.12f} \n average distance: {(p_d - n_d) / size} posi_ave: {p_d / size} nega_ave: {n_d / size}")

In [ ]:
print("start")
for t in range(EPOCHS):
    print(f"Epoch {t+1}\-------------------------------")
    test_loop(test_dataloader, model, loss_fn)
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

In [8]:
now = datetime.now()
formatted_date = now.strftime("%Y-%m-%d")
torch.save(model.state_dict(), f'C:/Users/yuuta/Documents/fashion/model_learning/versatility/models/triplet-image-{formatted_date}.pth')
